In [103]:
import codecs
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import numpy as np
from nltk import everygrams
from copy import copy
from tqdm import tqdm

## 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [104]:
with codecs.open("corpora\AnnaKarenina.txt", "r", "utf_8_sig") as file:
    corpus = file.read()
    
with codecs.open('corpora\WarAndPeace.txt', 'r', "utf_8_sig") as file:
    corpus += file.read()

In [105]:
len(corpus)

2531073

In [106]:
tokenizer = RegexpTokenizer(r'\w+')
abc = ' абвгдежзийклмнопрстуфхцчшщъыьэюя'

def tokenize(text, tokenizer=tokenizer):
    text = ''.join([c for c in text if c.lower() in abc])
    return ' '.join(tokenizer.tokenize(text.lower()))

def get_freqs(text, min_freq=0, n_gram=1):
    freqs = dict()
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        if value/len(text) > min_freq:
            freqs[key] = value/len(text)
    return freqs

def generate_mapping(freqs):
    original = list(freqs.keys())
    replacements = np.random.choice(original, replace=False, size=len(freqs))
    mapping = dict()
    for original_char, replacement_char in zip(original, replacements):
        mapping[original_char] = replacement_char
    return mapping


def apply_mapping(text, mapping):
    return ''.join([mapping.get(c, 'ь') for c in text])


def get_reverse_mapping(corpus_freqs, text_freqs):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    for text_char, text_freq in text_freqs_sorted:
        min_diff = 1
        best_char = None
        for corpus_char, corpus_freq in corpus_freqs_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff

        reverse_mapping[text_char] = best_char
        corpus_freqs_sorted = [(char, freq) for char, freq in corpus_freqs_sorted if char !=best_char]
        
    return reverse_mapping


def char_accuracy(text1, text2):
    assert len(text1) == len(text2)
    right_chars = 0
    for c1, c2 in zip(text1, text2):
        right_chars += int(c1 == c2)
    return right_chars / len(text1)

In [107]:
tokenized_corpus = tokenize(corpus)

In [108]:
corpus_freqs = get_freqs(tokenized_corpus)

In [109]:
mapping = generate_mapping(corpus_freqs)

In [110]:
text = tokenized_corpus[100:400]

In [111]:
encoded_text = apply_mapping(text, mapping)
text_freqs = get_freqs(encoded_text)
encoded_text

'шочмоъяк щвшовьикшаубуфкшюыхпшцмшюыхпмшсмфюмйшцвочмоъяк мйшовьийшцвочмоъяк мшауо увьхшзъушсцкпмшуш вчцщбштвццуоъйбшушядл кшуш вывшушовьившушчвяу вчвосуьшюуоъукцоъ вшяв шъуяоъугыуьмцшркыусупушющбмцкйчмоъишавы мйяв шъуяоъугмццмшсмывцкцмыуьмцшркыусупушющбмцкймццмшсмывцкцмшауымэкямшоу ывьвццксу ш овюцв '

In [112]:
reverse_mapping = get_reverse_mapping(corpus_freqs, text_freqs)
decoded_text = apply_mapping(encoded_text, reverse_mapping)
decoded_text

'отуатвкслчнотндзсобеьехсопржгоиаопржгаомахпаяоинтуатвкслаяотндзяоинтуатвкслаобетленджоэвеомисгаоеолнуичьоцнииетвяьоеокщфлсоеолнрноеотндзноеоункелнунтмедопетвеситвлнокнловектвешредаиоюсремегеопчьаисяуатвзобнрлаякнловектвешаииаомарнисиаредаиоюсремегеопчьаисяаииаомарнисиаобераъскаотелрндниисмелолтнпинл'

In [113]:
char_accuracy(text, decoded_text)

0.17333333333333334

## 2. Биграммы
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [114]:
corpus_freqs_bigram = get_freqs(tokenized_corpus, n_gram=2)
text_freqs_bigram = get_freqs(encoded_text, n_gram=2)

corpus_freqs_sorted = sorted(corpus_freqs_bigram.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs_bigram.items(), key=lambda x: x[1], reverse=True)

In [115]:
def get_reverse_mapping_bigram(corpus_freqs, text_freqs, init_mapping=None):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    if init_mapping is None:
        init_mapping = []
    reverse_mapping = {k: v for k, v in init_mapping}
    
    for i, (text_bigram, text_freq) in enumerate(text_freqs_sorted):

        filtred_freqs = copy(corpus_freqs_sorted)
        
        if text_bigram[0] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[0] == reverse_mapping[text_bigram[0]]]
            
        if text_bigram[1] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[1] == reverse_mapping[text_bigram[1]]]
              
        min_diff = 1
        best_bigram = None
        for bigram, freq in filtred_freqs:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_bigram = bigram
                min_diff = diff
                
        if text_bigram[0] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[0]] = best_bigram[0]
            
        if text_bigram[1] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[1]] = best_bigram[1]

        
    return reverse_mapping

In [116]:
reverse_mapping_bigram = get_reverse_mapping_bigram(corpus_freqs_bigram, text_freqs_bigram)
decoded_text = apply_mapping(encoded_text, reverse_mapping_bigram)
decoded_text

' о оо к яь  о ст  оо об  ся б ио ся бо ообсон и о оо к яон о стн и о оо к яо ооояо с  л о ои бо о я  иь  л ииоо н  о китя  о я я  о о ст  о   коя   ооос соо о ио я  к я  око отяосои б яоообо сь ои н оо т о яяонк я  око отоиио ооя и иояосои б яоообо сь ои ноиио ооя и ио оояоб ко оояя с ии ооя яо си я'

In [117]:
char_accuracy(text, decoded_text)

0.22666666666666666

## 3. MCMC-сэмплирование
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


In [118]:
def get_freqs_smooth(text, min_freq=0, n_gram=2):
    freqs = dict()
    vocab_len = len(set(text))**n_gram
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        freqs[key] = (value + 1) / (len(text) + vocab_len) # Сглаживаем, чтобы не было нулей
    return freqs

def get_text_proba(text, mapping, freqs, n_gram=2):
    decoded_text = apply_mapping(text, mapping)
    log_proba = 0
    for i in range(len(decoded_text) - n_gram):
        bigram = decoded_text[i: i + n_gram]
        bigram_proba = freqs.get(bigram)
        if bigram_proba is None:
            bigram_proba = 1 / (len(text) + len(abc)**n_gram) # Сглаживаем, чтобы не было нулей
            
        log_proba += np.log(bigram_proba)
    return log_proba
        
def get_reverse_mapping_mcmc(encoded_text, abc_encoded, abc_corpus, freqs_corpus, n_iters=10000, n_trials=10, n_gram=2):

    accept_count = 0
    best_mapping = None
    all_mappings = []
    best_log_likekihood = -np.inf

    for trial in tqdm(range(n_trials), leave=False, position=0, total=n_trials):

        abc_encoded = list(abc_encoded)
        abc_iter = list(abc_corpus)
        reverse_mapping = {k: v for k, v in zip(abc_encoded, abc_iter[:len(abc_encoded)])}
        log_proba_current = get_text_proba(encoded_text, reverse_mapping, freqs_corpus, n_gram=n_gram)

        for i in range(n_iters):
            abc_proposal = abc_iter[:]
            idx1, idx2 = np.random.choice(len(abc_proposal), replace=False, size=2)
            abc_proposal[idx1], abc_proposal[idx2] = abc_proposal[idx2], abc_proposal[idx1]
            reverse_mapping_proposal = {k: v for k, v in zip(abc_encoded, abc_proposal[:len(abc_encoded)])}
            log_proba_proposal = get_text_proba(encoded_text, reverse_mapping_proposal, freqs_corpus, n_gram=n_gram)

            p_accept = np.exp(log_proba_proposal - log_proba_current)

            if p_accept > np.random.rand():
                accept_count += 1
                abc_iter = abc_proposal
                log_proba_current = log_proba_proposal
                reverse_mapping = reverse_mapping_proposal

        if log_proba_current > best_log_likekihood:
            best_log_likekihood = log_proba_current
            best_mapping = reverse_mapping
            
        all_mappings.append(reverse_mapping)


    print(f'Best likelihood: {best_log_likekihood}')        
    print(f'Accept raito: {accept_count / (n_iters * n_trials)}')
    return best_mapping, all_mappings

In [119]:
freqs_corpus = get_freqs_smooth(tokenized_corpus, n_gram=2)

In [134]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc, 
    abc_corpus=abc,
    freqs_corpus=freqs_corpus,
    n_trials=10
)

Best likelihood: -1653.5418694166099
Accept raito: 0.02628


In [135]:
char_accuracy(apply_mapping(encoded_text, best_reverse_mapping), decoded_text)

0.23333333333333334

In [136]:
apply_mapping(encoded_text, best_reverse_mapping)

' счастривые сельи похожи здум на здума кажзая несчастривая селья несчастрива посвоелу это книма о вечных щенностях о ръфви о веде о селье о черовеческол зостоинстве рев торстогдолан бидокомо зыханиячасть педваярев торстоганна каденинадолан бидокомо зыханияанна каденина подашира совделенников всезнев'

## 4. Расшифруйте сообщение:

In [123]:
message = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [124]:
message_freqs = get_freqs(message, n_gram=1)

corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(message_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_message = ''.join([c for c, _ in message_freqs_sorted])
abc_corpus, abc_message

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', 'ႨდႹჂჵსႣჲჳႭშႩႼႧიႲხჾჰქჇჱႽჶეႠႾნ')

In [125]:
freqs_corpus = get_freqs_smooth(tokenized_corpus, n_gram=2)

In [126]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    message, 
    abc_encoded=abc_message, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus,
    n_iters=10000,
    n_trials=50,
)

Best likelihood: -1231.2446542698947
Accept raito: 0.043866


In [127]:
apply_mapping(message, best_reverse_mapping)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешащ'